<a href="https://colab.research.google.com/github/AgungPambudi/data-science/blob/main/polars_vs_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01) Polars

In [ ]:
# !pip install typing-extensions<4,>=3.7.4.3
! pip install polars

     |████████████████████████████████| 10.2 MB 16.1 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.0.1 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import time
import polars as pl

In [ ]:
nmin = 2
nmax=5000
# df = pd_create_random(nmax=5000000)
df = pd.DataFrame()
df['key'] = np.arange(0, nmax)
for i in range(0, nmin):
  df[ f'int{i}'] = np.random.randint(0, 100,size=(nmax, ))
  df[ f'flo{i}'] = np.random.rand(1, nmax)[0] 
  df[ f'str{i}'] =  [ ",".join([ str(t) for t in np.random.randint(10000000,999999999,size=(500, )) ] )  for k in range(0,nmax) ]
print(df.head)
df.to_parquet('myfile')

<bound method NDFrame.head of        key  int0  ...      flo1                                               str1
0        0    77  ...  0.532411  129572193,161379504,560007030,39409361,5646859...
1        1    60  ...  0.821610  479896937,166591274,22960819,955921722,2649981...
2        2    37  ...  0.187986  581724495,792153200,912463265,385820551,985867...
3        3    39  ...  0.960426  864878244,428722347,51383344,205167412,1806705...
4        4     7  ...  0.853511  856069926,639227736,966028760,912889966,505677...
...    ...   ...  ...       ...                                                ...
4995  4995    42  ...  0.246752  131334653,699001996,990319857,608624628,267397...
4996  4996    85  ...  0.841766  560652964,608204686,200347162,73818569,4842502...
4997  4997    10  ...  0.229511  134346108,634803278,583669340,616762479,927811...
4998  4998    45  ...  0.008502  859053825,18410487,130580246,48075785,76319930...
4999  4999    34  ...  0.558168  884843338,526767794,2558

In [ ]:
#### Pandas version
dfp = pd.read_parquet('myfile')
t0 = time.time()
for i in range(nmin):
  dfp['new1'] = dfp.apply(lambda x :  min( x['str1'].split(","))   , axis=1)
  dfp.groupby([f'int{i}']).agg({'count'})
  dfp.groupby(['key']).apply(lambda x:";".join(x[f'str{i}'].values))
  dfp.groupby([f'flo{i}']).agg({'sum'})
print(time.time() - t0)
dfp.to_parquet('myfile.parquet')

1.1416618824005127


In [ ]:
### POLARS Version 
df = pl.read_parquet('myfile')
t0 = time.time()
for i in range(nmin):
  df['new1'] = df.select(["*",  pl.col("str1").apply(lambda x : min(x.split(",")) ).alias("booknew")])['booknew']
  df.groupby(f'int{i}').agg(pl.all().count())
  df.groupby('key').agg(pl.col(f'str{i}')).select([pl.col('key'), pl.col(f'str{i}').arr.join(",")])
  df.groupby([f'flo{i}']).agg(pl.all().sum())
print(time.time() - t0)
df.to_parquet('myfile.parquet.polars')

0.511763334274292


### Let's compare our dataframes

In [ ]:
key = True
for j in dfp.columns:
  for i in range(nmax):
    if dfp[j][i] != df[j][i]:
      key = dfp[j][i], df[j][i]
      break
print(f'Our Pandas and Palars dataframe output are same: {key}')

Our Pandas and Palars dataframe output are same: True


# 02) Dict saving

In [ ]:
#  https://stackoverflow.com/questions/57983431/whats-the-most-space-efficient-way-to-compress-serialized-python-data

In [ ]:
 ! pip install blosc

     |████████████████████████████████| 2.6 MB 8.1 MB/s 


In [ ]:
import blosc
import numpy as np
import pickle

# data = np.random.rand(0, 3, 100000)

data = { str(i) :  ",".join([ str(t) for t in np.random.randint(10000000,999999999,size=(512, )) ] )     for i in range(0, 70000) }


In [ ]:


t0 = time.time()
pickled_data = pickle.dumps(data)  # returns data as a bytes objecttime
compressed_pickle = blosc.compress(pickled_data)

with open("test.dat", "wb") as f:
    f.write(compressed_pickle)
print(time.time()-t0)


t0 = time.time()
with open("test.dat", "rb") as f:
    compressed_pickle = f.read()

depressed_pickle = blosc.decompress(compressed_pickle)
data = pickle.loads(depressed_pickle)  # turn bytes object back into data
print(str(data)[:100] )
print(time.time()-t0)

1.5925381183624268
{'0': '839172569,57336951,132738278,485110074,884977886,643024554,415457049,322042677,300731604,6648
2.4512014389038086


In [ ]:
LINES = 10000
districts = np.random.choice(['A', 'B', 'C', 'D'], size=LINES, replace=True)
space = np.random.randint(1000, size=LINES)
price = np.random.randint(1000, size=LINES)
data = {
    "District": districts,
    "Space": space,
    "Price": price,
}


def pandas_test():
  df = pd.DataFrame(data)
  agg_df = df.groupby(["District"]).agg([min, np.average])
  apply_df = df.groupby("District").apply(lambda d: d.Space.sum() / d.Price.sum())
  df.to_parquet('pandas1.parquet')
  loaded_df = pd.read_parquet('pandas1.parquet')

  return df, agg_df, apply_df, loaded_df


def polars_test():
  df = pl.DataFrame(data)
  agg_df = df.groupby("District", maintain_order=True).agg([pl.mean("*"), pl.min("*")])
  apply_df = df.groupby("District", maintain_order=True).agg(
    pl.apply(
        f=lambda spacePrice: spacePrice[0].sum() / spacePrice[1].sum(),
        exprs=["Space", "Price"]
        )
  )
  df.to_parquet('polars1.parquet')
  loaded_df = pl.read_parquet('polars1.parquet')

  return df, agg_df, apply_df, loaded_df

pandas_result = pandas_test()
polars_result = polars_test()


print("Pandas:")
%timeit pandas_test()

print("Polars:")
%timeit polars_test()

Pandas:
The slowest run took 7.02 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 5: 12.6 ms per loop
Polars:
100 loops, best of 5: 5.26 ms per loop


## Pivoting Example



### Data generation

In [ ]:
import numpy as np
import pandas as pd
import polars as pl

N_ROWS = 50

event_sizes = np.random.randint(1,9, size=N_ROWS)
data = {
    'timestamp': np.arange(N_ROWS, dtype=np.float16) \
                    + np.random.randn(N_ROWS)+1,
    'detector_type': np.random.choice(['MM', 'sTGC'], N_ROWS),
    'sector_id': np.random.choice(np.arange(16)+1, N_ROWS),
    'PCB_no': [np.random.choice(np.arange(4)+1, n)
                    for n in event_sizes],
    'reading': [np.random.randint(1, 1025, size=n)
                for n in event_sizes]
}

df_pd = pd.DataFrame(data)
df_pl = pl.from_pandas(df_pd)

df_pl.head()

timestamp,detector_type,sector_id,PCB_no,reading
f64,str,i64,list,list
1.046554,"""sTGC""",4,"[2, 4, ... 3]","[683, 454, ... 248]"
1.706114,"""MM""",16,"[2, 1, ... 2]","[549, 918, ... 639]"
0.734288,"""MM""",5,"[1, 3, ... 4]","[456, 167, ... 527]"
3.193237,"""MM""",3,"[3, 1, ... 2]","[807, 193, ... 265]"
5.033244,"""MM""",8,"[3, 2, ... 1]","[517, 86, ... 376]"


### `pandas`

In [ ]:
%%time
# get the mean value of 'PCB_no' and 'reading' through exploding and pivoting
pivot_pd = df_pd.explode(['reading', 'PCB_no']).pivot_table(
            index=[ 'timestamp', 'detector_type', 'sector_id'], 
            values=['reading', 'PCB_no'], aggfunc=np.mean
        ).sort_values('timestamp').reset_index()
        
# get the total number of readings by detector type
readings_by_type_pd = df_pd.explode(['reading', 'PCB_no']).groupby('detector_type')[['reading']].count()\
    .rename(columns={'reading':'reading_count'}).sort_index()

# convert the pivot table to other types
# python dictionary, arrow table, numpy array and json
dict_pd = pivot_pd.to_dict()
numpy_pd = pivot_pd.to_numpy()
json_pd = pivot_pd.to_json()

CPU times: user 62 ms, sys: 756 µs, total: 62.8 ms
Wall time: 77.6 ms


### `polars`

In [ ]:
%%time
# there is no `pivot_table` in polars
# we use `melt` and `pivot` instead
pivot_pl = df_pl.explode(['reading', 'PCB_no']).melt(
                id_vars=['timestamp', 'detector_type', 'sector_id'], 
                value_vars=['PCB_no','reading']
            ).pivot(index=[ 'timestamp','detector_type', 'sector_id'], 
                columns='variable', values='value', 
                aggregate_fn='mean') \
        .sort('timestamp')

# for the readings by type we can use the same pandas syntax (sort of)
# this comes with the price of a DeprecationWarning, but no speed compromise
readings_by_type_pl = df_pl.explode(['reading', 'PCB_no']).groupby(
                "detector_type", maintain_order=True
                )[['reading']].count().sort('detector_type')

# convert the pivot table to other types
# python dictionary, numpy array and json string
dict_pl = pivot_pl.to_dict()
numpy_pl = pivot_pl.to_numpy()
json_pl = pivot_pl.to_json()

accessing GroupBy by index is deprecated, consider using the `.agg` method
CPU times: user 4.63 ms, sys: 944 µs, total: 5.58 ms
Wall time: 5.66 ms


#### comparison

In [ ]:
print(readings_by_type_pd, readings_by_type_pl)

               reading_count
detector_type               
MM                       107
sTGC                     144 shape: (2, 2)
┌───────────────┬───────────────┐
│ detector_type ┆ reading_count │
│ ---           ┆ ---           │
│ str           ┆ u32           │
╞═══════════════╪═══════════════╡
│ MM            ┆ 107           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ sTGC          ┆ 144           │
└───────────────┴───────────────┘


In [ ]:
print(pivot_pd.head(), pivot_pl.head())

   timestamp detector_type  sector_id  PCB_no  reading
0   0.734288            MM          5   1.875  363.125
1   1.046554          sTGC          4   2.800  617.800
2   1.706114            MM         16   2.400  645.800
3   3.193237            MM          3   2.000  532.400
4   3.792488            MM          4   2.000  943.000 shape: (5, 5)
┌───────────┬───────────────┬───────────┬────────┬─────────┐
│ timestamp ┆ detector_type ┆ sector_id ┆ PCB_no ┆ reading │
│ ---       ┆ ---           ┆ ---       ┆ ---    ┆ ---     │
│ f64       ┆ str           ┆ i64       ┆ f64    ┆ f64     │
╞═══════════╪═══════════════╪═══════════╪════════╪═════════╡
│ 0.734288  ┆ MM            ┆ 5         ┆ 1.875  ┆ 363.125 │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ 1.046554  ┆ sTGC          ┆ 4         ┆ 2.8    ┆ 617.8   │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ 1.706114  ┆ MM            ┆ 16        ┆ 2.4    ┆ 645.8   │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌

In [ ]:
# check if the numpy represtentations are identical
assert( (numpy_pd != numpy_pl).sum() == 0)